In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats.mstats import winsorize

In [106]:
# Opt in to future behavior to avoid downcasting warnings
pd.set_option('future.no_silent_downcasting', True)
INPUT_FILE = 'masked_kiva_loans.csv'
OUTPUT_FILE = 'kiva_loans_cleaned_ml_ready.csv'

In [107]:
print("Loading data...")
df = pd.read_csv(INPUT_FILE, low_memory=False)
print(f"Initial shape: {df.shape}\n")

Loading data...
Initial shape: (33561, 11)



In [108]:
# 1. Initial Assessment
print("Initial missing values:")
print(df.isnull().sum().sort_values(ascending=False)[:10], "\n")


Initial missing values:
partner_id            634
borrower_genders      195
id                      0
loan_amount             0
funded_amount           0
country                 0
sector                  0
term_in_months          0
lender_count            0
repayment_interval      0
dtype: int64 



# 3. Data Cleaning
# 3.1 Drop duplicate rows if any

In [109]:
print("Dropping duplicates...")
prev_size = len(df)
df.drop_duplicates(inplace=True)
print(f"Removed {prev_size - len(df)} duplicates\n")

Dropping duplicates...
Removed 0 duplicates



In [110]:
# 3. Column Selection (Keep only relevant columns)
keep_cols = [
    'id', 'funded_amount', 'loan_amount', 'sector', 'country',
    'partner_id', 'term_in_months', 'lender_count', 'borrower_genders',
    'repayment_interval', 'date'
]
df = df[keep_cols]

# 3.2 Handle missing values
# - If columns have too many missing values, consider dropping
# - Otherwise impute or fill

In [111]:
print("Handling missing values:")

Handling missing values:


#  Process borrower_genders column: count number of female and male

In [112]:
df['borrower_genders'] = df['borrower_genders'].str.lower().fillna('unknown')
df['female_count'] = df['borrower_genders'].str.split(',').apply(
    lambda x: sum('female' in g.strip() for g in x) if isinstance(x, list) else 0)
df['male_count'] = df['borrower_genders'].str.split(',').apply(
    lambda x: sum('male' in g.strip() for g in x) if isinstance(x, list) else 0)
df['total_borrowers'] = df['female_count'] + df['male_count']

# For numerical columns, fill with median

In [113]:
num_cols = ['funded_amount', 'loan_amount', 'term_in_months', 'lender_count']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())
    # Convert to appropriate numeric types
    if col in ['lender_count', 'term_in_months']:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    else:
        df[col] = pd.to_numeric(df[col], downcast='float')

# For categorical columns, fill with mode

In [114]:
cat_cols = ['sector', 'country', 'repayment_interval']
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0]).astype('category')

# 3.3 Convert date columns

In [115]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df[~df['date'].isna()]  # Remove rows with invalid dates

# 3.4 Remove or cap outliers (example using quantiles)

In [116]:
df['partner_id'].duplicated().sum()

np.int64(33279)

In [117]:
df.drop(columns=['partner_id'], inplace=True)

In [118]:
Q1 = df['funded_amount'].quantile(0.25)
Q3 = df['funded_amount'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['funded_amount'] < lower_bound) | (df['funded_amount'] > upper_bound)]
df['funded_amount'] = np.where(df['funded_amount'] > upper_bound, upper_bound, df['funded_amount'])
df['funded_amount'] = np.where(df['funded_amount'] < lower_bound, lower_bound, df['funded_amount'])


In [119]:
q1 = df['loan_amount'].quantile(0.25)
q3 = df['loan_amount'].quantile(0.75)
iqr = q3 - q1
lowerbound = q1 - 1.5 * iqr
upperbound = q3 + 1.5 * iqr

outlier = df[(df['loan_amount'] < lowerbound) | (df['loan_amount'] > upperbound)]
df['loan_amount'] = np.where(df['loan_amount'] > upperbound, upperbound, df['loan_amount'])
df['loan_amount'] = np.where(df['loan_amount'] < lowerbound, lowerbound, df['loan_amount'])

In [120]:
Q2 = df['term_in_months'].quantile(0.25)
Q4 = df['term_in_months'].quantile(0.75)
IQR2 = Q4 - Q2
lower = Q2 - 1.5 * IQR2
upper = Q4 + 1.5 * IQR2

outl = df[(df['term_in_months'] < lower) | (df['term_in_months'] > upper)]
df['term_in_months'] = np.where(df['term_in_months'] > upper, upper, df['term_in_months'])
df['term_in_months'] = np.where(df['term_in_months'] < lower, lower, df['term_in_months'])

In [121]:
q2 = df['lender_count'].quantile(0.25)
q4 = df['lender_count'].quantile(0.75)
iqr2 = q4 - q2
low = q2 - 1.5 * iqr2
upp = q4 + 1.5 * iqr2

out = df[(df['lender_count'] < low) | (df['lender_count'] > upp)]
df['lender_count'] = np.where(df['lender_count'] > upp, upp, df['lender_count'])
df['lender_count'] = np.where(df['lender_count'] < low, low, df['lender_count'])

# 4. Data Transformation
# 4.1 Feature engineering: extract date parts

In [122]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['year_month'] = df['date'].dt.to_period('M')

In [123]:
df['funding_ratio'] = df['funded_amount'] / df['loan_amount'].replace(0, np.nan)
df['amount_per_lender'] = df['funded_amount'] / df['lender_count'].replace(0, np.nan)

In [124]:
# Gender features
df['female_only'] = (df['female_count'] > 0) & (df['male_count'] == 0)
df['male_only'] = (df['male_count'] > 0) & (df['female_count'] == 0)
df['mixed_gender'] = (df['female_count'] > 0) & (df['male_count'] > 0)

# 5. Final Exploration

In [125]:
# 7. Final Validation
print("\nFinal validation:")
print(f"Final shape: {df.shape}")
print("Missing values check:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Final validation:
Final shape: (33561, 21)
Missing values check:
amount_per_lender    152
dtype: int64


In [126]:
# Check data types
print("\nData types:")
print(df.dtypes)


Data types:
id                             int64
funded_amount                float64
loan_amount                  float64
sector                      category
country                     category
term_in_months               float64
lender_count                 float64
borrower_genders              object
repayment_interval          category
date                  datetime64[ns]
female_count                   int64
male_count                     int64
total_borrowers                int64
year                           int32
month                          int32
year_month                 period[M]
funding_ratio                float64
amount_per_lender            float64
female_only                     bool
male_only                       bool
mixed_gender                    bool
dtype: object


In [127]:
# Basic stats
print("\nNumerical columns summary:")
print(df[num_cols].describe())


Numerical columns summary:
       funded_amount   loan_amount  term_in_months  lender_count
count    33561.00000  33561.000000    33561.000000  33561.000000
mean       642.65889    691.773934       12.683889     17.448854
std        525.92744    567.818345        5.061591     14.193749
min          0.00000     25.000000        1.000000      0.000000
25%        250.00000    275.000000        8.000000      7.000000
50%        450.00000    500.000000       13.000000     13.000000
75%        900.00000    975.000000       14.000000     24.000000
max       1875.00000   2025.000000       23.000000     49.500000


# 6. Save cleaned data

In [128]:
df.to_csv('kiva_loans_cleaned_final.csv', index=False)
print("Final shape:", df.shape)
print("Data cleaned and saved as 'kiva_loans_cleaned_final.csv'.")

Final shape: (33561, 21)
Data cleaned and saved as 'kiva_loans_cleaned_final.csv'.
